In [19]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from pathlib import Path
from sklearn.model_selection import train_test_split

import tensorflow as tf
import os
from sklearn.metrics import confusion_matrix, classification_report

In [20]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
)

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [21]:
train_df="/kaggle/input/surface-crack/train"
test_df="/kaggle/input/surface-crack/test"
valid_df="/kaggle/input/surface-crack/valid"

In [22]:
train_data = train_gen.flow_from_directory(
    train_df,
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True,

 
)

val_data = train_gen.flow_from_directory(
    valid_df,
    target_size=(240, 240),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True

   

)

test_data = train_gen.flow_from_directory(
    test_df,
    target_size=(240, 240),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=False,

    
)

Found 600 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [23]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,RMSprop, Adagrad

from keras.layers import BatchNormalization

In [24]:
 def random_invert_img(x, p=0.6):
    if tf.random.uniform([]) < p:
        x = (255 - x)
    else:
        x
    return x
 def random_inv(factor = 0.65):
    return tf.keras.layers.Lambda(lambda x:random_invert_img(x, factor)) 

In [35]:
inputs = tf.keras.Input(shape=(240, 240, 3))
x=tf.keras.layers.RandomContrast(0.2,0.7)
x=tf.keras.layers.RandomCrop(120,120)
x=tf.keras.layers.RandomRotation(0.2)
x=tf.keras.layers.RandomFlip("horizontal_and_vertical")
x=tf.keras.layers.RandomHeight(0.3)
x=random_inv()





x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.BatchNormalization()(x)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print(model.summary())

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 240, 240, 3)]     0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 238, 238, 32)      896       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 119, 119, 32)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 117, 117, 64)      18496     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 58, 58, 64)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 56, 56, 128)       73856     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 28, 28, 128)       0   

In [27]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


In [28]:
filepath="we111ights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
es = EarlyStopping(monitor='val_loss', patience=5,restore_best_weights=True)
callbacks_list = [checkpoint, es]

In [36]:


history = model.fit(train_data,validation_data=val_data,epochs = 100,callbacks=callbacks_list ,verbose=1)

Epoch 1/100
19/19 [==============================] - 3s 103ms/step - loss: 0.6022 - accuracy: 0.6600 - val_loss: 0.6789 - val_accuracy: 0.5000

Epoch 00001: val_loss did not improve from 0.15290
Epoch 2/100
19/19 [==============================] - 2s 105ms/step - loss: 0.4795 - accuracy: 0.7633 - val_loss: 0.7062 - val_accuracy: 0.5000

Epoch 00002: val_loss did not improve from 0.15290
Epoch 3/100
19/19 [==============================] - 2s 98ms/step - loss: 0.4427 - accuracy: 0.7950 - val_loss: 0.6605 - val_accuracy: 0.5000

Epoch 00003: val_loss did not improve from 0.15290
Epoch 4/100
19/19 [==============================] - 2s 112ms/step - loss: 0.4257 - accuracy: 0.7950 - val_loss: 0.6838 - val_accuracy: 0.5000

Epoch 00004: val_loss did not improve from 0.15290
Epoch 5/100
19/19 [==============================] - 2s 90ms/step - loss: 0.3952 - accuracy: 0.8233 - val_loss: 0.7599 - val_accuracy: 0.5000

Epoch 00005: val_loss did not improve from 0.15290
Epoch 6/100
19/19 [========

In [37]:
fig = px.line( 
    history.history,
    y=['loss', 'val_loss'],
    labels={'index': "Epoch", 'value': "Loss"},
)

fig.show()

In [38]:
def evaluate_model(model, val_data):
    
    results = model.evaluate(val_data, verbose=0)
    loss = results[0]
    acc = results[1]
    print("    Test Loss: {:.5f}".format(loss))
    print("Test Accuracy: {:.2f}%".format(acc * 100))
    y_pred = np.squeeze((model.predict(test_data) >= 0.5).astype(np.int))
   
    clr = classification_report(test_data.labels, y_pred, target_names=["NEGATIVE", "POSITIVE"])
    print("Classification Report:\n----------------------\n", clr)
                                                                        



**classifcation report for Test data**

In [39]:
evaluate_model(model, test_data)

    Test Loss: 0.22286
Test Accuracy: 92.00%
Classification Report:
----------------------
               precision    recall  f1-score   support

    NEGATIVE       0.86      1.00      0.93       100
    POSITIVE       1.00      0.84      0.91       100

    accuracy                           0.92       200
   macro avg       0.93      0.92      0.92       200
weighted avg       0.93      0.92      0.92       200



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



**classifcation report for validation data**

In [40]:
evaluate_model(model, val_data)

    Test Loss: 0.13683
Test Accuracy: 93.50%
Classification Report:
----------------------
               precision    recall  f1-score   support

    NEGATIVE       0.86      1.00      0.93       100
    POSITIVE       1.00      0.84      0.91       100

    accuracy                           0.92       200
   macro avg       0.93      0.92      0.92       200
weighted avg       0.93      0.92      0.92       200



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



In [41]:
y_pred.size

NameError: name 'y_pred' is not defined